In [1]:
!pip install genpipes

You should consider upgrading via the 'C:\Users\Jayra\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip' command.


In [2]:
import string

import xml.etree.ElementTree as ET
import pandas as pd
from nltk.tokenize import word_tokenize, sent_tokenize  
import os

In [13]:
tags = ["I-NAME", "I-PROFESSION", "I-LOCATION", "I-AGE", "I-DATE", "I-CONTACT", "I-ID", "I-PHI",  "O"]
class Tags():
    def __init__(self):
        self.entities = []
        self.tags = []
        self.sub_tags = [] 
    
    def append_entity(self, entity: list):
        self.entities.extend(entity)
    
    def append_tag(self, tag: list):
        self.tags.extend(tag)
    
    def append_sub_tags(self, sub_tag: list):
        self.sub_tags.extend(sub_tag)
    

def gather_tags(root):
    entities = []
    tags = []
    sub_tags = []
    for child in root.iter('TAGS'):
        for subchild in child:
            entity = subchild.get("text").split(" ")
            [tags.append(subchild.tag) for ent in entity]
            [sub_tags.append(subchild.get("TYPE")) for ent in entity]
            entities.extend(entity)
    
    tags.append("O")
    return entities, tags, sub_tags

def split_to_sent(root):
    doc_txt = str(root.find('TEXT').text)
    tokens = sent_tokenize(doc_txt)

    return tokens

def split_to_tokens(text):
    tokens = word_tokenize(text, preserve_line=True)

    return tokens

def label_txt(root):
    new_tags = ["I-NAME", "I-PROFESSION", "I-LOCATION", "I-AGE", "I-DATE", "I-CONTACT", "I-ID", "I-PHI",  "O"]
    orig_tags = ["NAME", "PROFESSION", "LOCATION", "AGE", "DATE", "CONTACT", "ID", "PHI", "O"]
    entities, tags, sub_tags = gather_tags(root)
    
    sentences = split_to_sent(root)
    ents, toks = [], []
    
    for sentence in sentences:
        tokens = split_to_tokens(sentence)
        for token in tokens:
            idx = orig_tags.index(tags[entities.index(token) if token in entities else -1])
            ents.append(token) 
            toks.append(new_tags[idx])

        ents.append("")
        toks.append("")
    
    return ents, toks

     

        



In [8]:
listOfFile = os.listdir("./data/test/")
listOfFileDev = listOfFile[:len(listOfFile)//2]
listOfFileTest = listOfFile[len(listOfFile)//2:]
"""
ner_tags_dev = Tags()
for file in listOfFileDev:
    tree = ET.parse(f"./data/test/{file}")
    root = tree.getroot()
    label_txt(root, "dev.txt")
"""
    

'\nner_tags_dev = Tags()\nfor file in listOfFileDev:\n    tree = ET.parse(f"./data/test/{file}")\n    root = tree.getroot()\n    label_txt(root, "dev.txt")\n'

In [20]:
"""ner_tags = Tags()
for file in listOfFileTest:
    tree = ET.parse(f"./data/test/{file}")
    root = tree.getroot()
    label_txt(root, "test.txt")"""

listOfFile = os.listdir("./data/test/")
listOfFileDev = listOfFile[:len(listOfFile)//2]
listOfFileTest = listOfFile[len(listOfFile)//2:]

entities, tokens = [], []
for file in listOfFileTest:
    tree = ET.parse(f"./data/test/{file}")
    root = tree.getroot()
    ents, toks = label_txt(root)
    entities.extend(ents)
    tokens.extend(toks) 

new_tokens = [entities, tokens]
df = pd.DataFrame(new_tokens).transpose()
print(df)
    

                 0           1
0           Record           O
1             date           O
2                :           O
3       2077-03-31      I-DATE
4            Notre  I-LOCATION
...            ...         ...
206265           4           O
206266      Beeper           O
206267           #           O
206268       07736   I-CONTACT
206269                        

[206270 rows x 2 columns]


In [21]:
df.to_csv("test.txt", sep=" ", index=False, columns=None)